In [17]:
from datasets import load_dataset
from sae_lens import SAE
from transformer_lens import HookedTransformer
from transformers import AutoModel, AutoModelForCausalLM, AutoTokenizer

### Load in SAE

In [9]:
sae, cfg, _ = SAE.from_pretrained(
    release="gemma-2b-it-res-jb",
    sae_id="blocks.12.hook_resid_post",
)


In [13]:
hook = sae.cfg.hook_name

### Load in Hooked Model

In [ ]:
model = HookedTransformer.from_pretrained('google/gemma-2b-it')

#### Load in Tokenizer

In [18]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
def tokenize_batch(batch):
    return tokenizer(
        batch["input"],
        truncation=True
        max_length=256,
        padding=False
    )

### Load in Data

In [ ]:
dataset = load_dataset("facebook/kilt_tasks", "eli5", split="train")


Load in Question
Tokenizer it
Pass through model
get activation
Encode it 

In [ ]:
tokens = model.to_tokens(text, prepend_bos=True).to(device)

In [ ]:
with torch.no_grad():
    _, cache = model.run_with_cache(tokens, names_filter=hook)
acts = cache[hook]
feats = sae.encode(acts)
feats_sum = feats.mean(dim=1)